# DECISION TREE

### INSERTAMOS PAQUETES IMPORTANTES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas_profiling

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

# tamaño de la figura en pulgadas
rcParams["figure.figsize"] = 10, 6
np.random.seed(42)

C:\Users\garro\miniconda3\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\garro\miniconda3\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\garro\AppData\Local\Temp\ipykernel_50072\3713271866.py:12: DeprecationWarning: `import pandas_profiling` is going to be deprecated by Apri

In [2]:
#IMPORTAMOS BASE DE DATOS 1
bd=pd.read_excel(r'C:\Users\garro\OneDrive\Escritorio\DATA SCIENCE\TRABAJO PRACTICO\ViolenciaGenero2.3.xlsx', sheet_name='casos')

#CAMBIAMOS FORMATO DE LA COLUMNA FECHA A 'DATETIME'
bd['FECHA']=pd.to_datetime(bd.FECHA, errors='coerce')

#LIMPIEZA DE VALORES NULOS. REEMPLAZO POR MEDIA
bd['EDAD']= bd['EDAD'].fillna(method='bfill')

#REMOCION DE OUTLIERS
def remove_outliers(df, column_name):
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df = df.loc[(df[column_name]>=lower_bound) & (df[column_name] <= upper_bound)]
    return df
bd = remove_outliers(bd, 'EDAD')
bd.head(10)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,VIOLENCIA_LABORAL,VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA,VIOLENCIA_OTRAS,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
5,2020-01-01,Buenos Aires,Mujer,23.0,Argentina,1,1,0,1,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
6,2020-01-01,Santiago Del Estero,Mujer,25.0,Paraguaya,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
7,2020-01-01,Santa Fe,Mujer,24.0,Argentina,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
8,2020-01-01,Santa Fe,Mujer,21.0,Argentina,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
9,2020-01-01,San Juan,Mujer,21.0,Argentina,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1


### PRUEBA CON GET.DUMMIES

In [5]:
# Paso 2: Dividir los datos en características (X) y variable objetivo (y)
X=bd.drop(['FECHA','VIOLENCIA_FISICA', 'VIOLENCIA_PSICOLOGICA', 'TIPOVIOLENCIA', 'CASO',
       'VIOLENCIA_SEXUAL', 'VIOLENCIA_ECONOMICA', 'VIOLENCIA_SIMBOLICA',
       'VIOLENCIA_DOMESTICA', 'VIOLENCIA_INSTITUCIONAL', 'VIOLENCIA_LABORAL',
       'VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA', 'VIOLENCIA_OTRAS','TiPO_VIOLENCIA'], axis=1)
y=bd['TiPO_VIOLENCIA']
X

,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA
0,Tucumán,Mujer,39.0,NaN,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
1,Buenos Aires,Mujer,39.0,NaN,Ex pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
2,Buenos Aires,Mujer,22.0,Argentina,Ex pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
3,Buenos Aires,Mujer,53.0,NaN,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
4,Buenos Aires,Mujer,38.0,Argentina,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
...,...,...,...,...,...,...,...,...
73488,Buenos Aires,Mujer,37.0,NaN,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS
73489,Buenos Aires,Mujer,53.0,Argentina,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_SEXUAL
73491,Buenos Aires,Mujer,52.0,Argentina,Ex pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_SEXUAL
73492,Buenos Aires,Mujer,29.0,NaN,Pareja,Varon,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS


In [6]:
# Paso 3: Convertir variables categóricas en variables dummy (one-hot encoding)
X = pd.get_dummies(X)

In [7]:
#Separamos en train y test!
(X_train, X_test,y_train, y_test) = train_test_split(X,y,stratify=y,test_size=0.30,random_state=42)

In [8]:
from sklearn.tree import DecisionTreeClassifier

#Creamos un arbol de decisión sencillo y lo fiteamos
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [9]:
y_test_pred = tree.predict(X_test) #Prediccion en Test

### MATRIZ DE CONFUSION

In [10]:
###Completar
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred) 

array([[12529,  1817],
       [ 5204,  1900]], dtype=int64)

### ACCURACY SCORE (nivel de precisión de predecir los resultados)

In [11]:
###Completar
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_test_pred)

0.6726806526806527

### RECALL SCORE (capacidad de un modelo para encontrar todos los ejemplos positivos)

In [12]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_test_pred, pos_label='VIOLENCIA_PSICOLOGICA')
recall

0.26745495495495497

### PRECISION SCORE (proporción de ejemplos positivos correctamente identificados con respecto al total de ejemplos identificados como positivos)

In [13]:
###Completar
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_test_pred, pos_label='VIOLENCIA_PSICOLOGICA')
precision

0.511164917944579

### F1

In [14]:
###Completar
from sklearn.metrics import f1_score
f1_score(y_test, y_test_pred, pos_label='VIOLENCIA_PSICOLOGICA') 

0.35116902319563814

### REPORTE

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

                       precision    recall  f1-score   support

     VIOLENCIA_FISICA       0.71      0.87      0.78     14346
VIOLENCIA_PSICOLOGICA       0.51      0.27      0.35      7104

             accuracy                           0.67     21450
            macro avg       0.61      0.57      0.57     21450
         weighted avg       0.64      0.67      0.64     21450



### PARA MEJORAR LA PERFORMANCE DEL ALGORITMO, HAREMOS UNA MAYOR LIMPIEZA DE DATOS Y LOS MODIFICAREMOS PARA QUE SEAN NUMERICOS Y NO CATEGORICOS

In [16]:
# REALIZAMOS UNA COPIA DE LA COLUMNA PROVINCIA PARA PRESERVAR LOS DATOS ORIGNALES.
bd['TIPO_VIOLENCIA_NUM'] = bd['TiPO_VIOLENCIA']

bd['TIPO_VIOLENCIA_NUM'] = bd['TIPO_VIOLENCIA_NUM'].replace('VIOLENCIA_FISICA', '0')
bd['TIPO_VIOLENCIA_NUM'] = bd['TIPO_VIOLENCIA_NUM'].replace('VIOLENCIA_PSICOLOGICA', '1')

bd.head(5)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA,VIOLENCIA_OTRAS,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA,TIPO_VIOLENCIA_NUM
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,0,0,Pareja,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0,1
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0


In [17]:
# REALIZAMOS UNA COPIA DE LA COLUMNA PROVINCIA PARA PRESERVAR LOS DATOS ORIGNALES.
bd['PROVINCIA_NUM'] = bd['PROVINCIA']

bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Ciudad Autónoma de Buenos Aires', '0')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Buenos Aires', '1')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Córdoba', '2')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Entre Ríos', '3')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('La Pampa', '4')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Santa Fe', '5')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Catamarca', '6')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Jujuy', '7')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('La Rioja', '8')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Salta', '9')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Santiago del Estero', '10')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Tucumán', '11')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Misiones', '12')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Chaco', '13')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Formosa', '14')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Corrientes', '15')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Mendoza', '16')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('San Luis', '17')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('San Juan', '18')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Chubut', '19')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Neuquén', '20')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Río Negro', '21')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Santa Cruz', '22')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Tierra del Fuego, Antártida e Islas del Atlántico Sur', '23')
bd['PROVINCIA_NUM'] = bd['PROVINCIA_NUM'].replace('Santiago Del Estero', '10')
bd.head(5)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,VIOLENCIA_OTRAS,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA,TIPO_VIOLENCIA_NUM,PROVINCIA_NUM
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,11
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,0,Pareja,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0,1,1
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1


In [18]:
# REALIZAMOS UNA COPIA DE LA COLUMNA PROVINCIA PARA PRESERVAR LOS DATOS ORIGNALES.
bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA']

bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA_NUM'].replace('Pareja', '0')
bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA_NUM'].replace('Ex pareja', '1')
bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA_NUM'].replace('Padre, madre, tutor u otro familiar', '2')
bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA_NUM'].replace('Superior jerarquico', '3')
bd['VINCULO_PERSONA_AGRESORA_NUM'] = bd['VINCULO_PERSONA_AGRESORA_NUM'].replace('Otro', '4')

bd.head(5)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA,TIPO_VIOLENCIA_NUM,PROVINCIA_NUM,VINCULO_PERSONA_AGRESORA_NUM
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,11,0
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,Pareja,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0,1,1,0
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,0


In [19]:
# REALIZAMOS UNA COPIA DE LA COLUMNA PROVINCIA PARA PRESERVAR LOS DATOS ORIGNALES.
bd['LUGAR_VIOLENCIA_NUM'] = bd['LUGAR_VIOLENCIA']

bd['LUGAR_VIOLENCIA_NUM'] = bd['LUGAR_VIOLENCIA_NUM'].replace('VIOLENCIA_INSTITUCIONAL', '0')
bd['LUGAR_VIOLENCIA_NUM'] = bd['LUGAR_VIOLENCIA_NUM'].replace('VIOLENCIA_LABORAL', '1')
bd['LUGAR_VIOLENCIA_NUM'] = bd['LUGAR_VIOLENCIA_NUM'].replace('VIOLENCIA_DOMESTICA', '2')

bd.head(5)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA,TIPO_VIOLENCIA_NUM,PROVINCIA_NUM,VINCULO_PERSONA_AGRESORA_NUM,LUGAR_VIOLENCIA_NUM
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,11,0,2
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1,2
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1,2
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0,1,1,0,2
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,0,2


In [20]:
# REALIZAMOS UNA COPIA DE LA COLUMNA PROVINCIA PARA PRESERVAR LOS DATOS ORIGNALES.
bd['MOTIVO_VIOLENCIA_NUM'] = bd['MOTIVO_VIOLENCIA']

bd['MOTIVO_VIOLENCIA_NUM'] =bd['MOTIVO_VIOLENCIA_NUM'].replace('VIOLENCIA_OTRAS', '0')
bd['MOTIVO_VIOLENCIA_NUM'] =bd['MOTIVO_VIOLENCIA_NUM'].replace('VIOLENCIA_ECONOMICA', '1')
bd['MOTIVO_VIOLENCIA_NUM'] =bd['MOTIVO_VIOLENCIA_NUM'].replace('VIOLENCIA_SEXUAL', '2')
bd['MOTIVO_VIOLENCIA_NUM'] =bd['MOTIVO_VIOLENCIA_NUM'].replace('VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA', '3')

bd.head(5)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA,TIPO_VIOLENCIA_NUM,PROVINCIA_NUM,VINCULO_PERSONA_AGRESORA_NUM,LUGAR_VIOLENCIA_NUM,MOTIVO_VIOLENCIA_NUM
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,11,0,2,0
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1,2,0
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,1,2,0
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0,1,1,0,2,0
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1,0,1,0,2,0


In [21]:
bd.columns

Index(['FECHA', 'PROVINCIA', 'GENERO_PERSONA_SIT_VIOLENCIA', 'EDAD',
       'PAIS_NACIMIENTO', 'VIOLENCIA_FISICA', 'VIOLENCIA_PSICOLOGICA',
       'VIOLENCIA_SEXUAL', 'VIOLENCIA_ECONOMICA', 'VIOLENCIA_SIMBOLICA',
       'VIOLENCIA_DOMESTICA', 'VIOLENCIA_INSTITUCIONAL', 'VIOLENCIA_LABORAL',
       'VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA', 'VIOLENCIA_OTRAS',
       'VINCULO_PERSONA_AGRESORA', 'GENERO_AGRESOR', 'CASO', 'TiPO_VIOLENCIA',
       'LUGAR_VIOLENCIA', 'MOTIVO_VIOLENCIA', 'TIPOVIOLENCIA',
       'TIPO_VIOLENCIA_NUM', 'PROVINCIA_NUM', 'VINCULO_PERSONA_AGRESORA_NUM',
       'LUGAR_VIOLENCIA_NUM', 'MOTIVO_VIOLENCIA_NUM'],
      dtype='object')

In [31]:
# Paso 2: Dividir los datos en características (X) y variable objetivo (y)
X=bd.drop(['FECHA', 'PROVINCIA', 'GENERO_PERSONA_SIT_VIOLENCIA', 'PAIS_NACIMIENTO', 
           'VIOLENCIA_FISICA', 'VIOLENCIA_PSICOLOGICA', 'TIPOVIOLENCIA',
       'VIOLENCIA_SEXUAL', 'VIOLENCIA_ECONOMICA', 'VIOLENCIA_SIMBOLICA',
       'VIOLENCIA_DOMESTICA', 'VIOLENCIA_INSTITUCIONAL', 'VIOLENCIA_LABORAL',
       'VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA', 'VIOLENCIA_OTRAS',
       'VINCULO_PERSONA_AGRESORA', 'GENERO_AGRESOR', 'CASO', 'TiPO_VIOLENCIA',
       'LUGAR_VIOLENCIA', 'MOTIVO_VIOLENCIA', 'TIPO_VIOLENCIA_NUM'], axis=1)
y=bd['TIPO_VIOLENCIA_NUM']
X.head(10)

,EDAD,PROVINCIA_NUM,VINCULO_PERSONA_AGRESORA_NUM,LUGAR_VIOLENCIA_NUM,MOTIVO_VIOLENCIA_NUM
0,39.0,11,0,2,0
1,39.0,1,1,2,0
2,22.0,1,1,2,0
3,53.0,1,0,2,0
4,38.0,1,0,2,0
5,23.0,1,1,2,1
6,25.0,10,0,2,1
7,24.0,5,1,2,0
8,21.0,5,0,2,1
9,21.0,18,0,2,1


In [32]:
X.PROVINCIA_NUM.isna().sum() 


0

In [33]:
X.VINCULO_PERSONA_AGRESORA_NUM.isna().sum() 

0

In [34]:
#Separamos en train y test!
(X_train, X_test,y_train, y_test) = train_test_split(X,y,stratify=y,test_size=0.30,random_state=42)

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
#Creamos un arbol de decisión sencillo y lo fiteamos
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [37]:
y_test_pred = tree.predict(X_test) #Prediccion en Test

### ACCURACY SCORE (nivel de precisión de predecir los resultados)

In [38]:
###Completar
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_test_pred)

0.6777622377622378

### MATRIZ DE CONFUSION

In [39]:
###Completar
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred) 

array([[12922,  1424],
       [ 5488,  1616]], dtype=int64)

### PRECISION SCORE (proporción de ejemplos positivos correctamente identificados con respecto al total de ejemplos identificados como positivos)

In [40]:
###Completar
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_test_pred, pos_label='1')
precision 

0.531578947368421

### RECALL SCORE (capacidad de un modelo para encontrar todos los ejemplos positivos)

In [41]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_test_pred, pos_label='1')
recall

0.22747747747747749

### F1

In [42]:
###Completar
from sklearn.metrics import f1_score
f1_score(y_test, y_test_pred, pos_label='1') 

0.3186119873817035

### REPORTE

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.70      0.90      0.79     14346
           1       0.53      0.23      0.32      7104

    accuracy                           0.68     21450
   macro avg       0.62      0.56      0.55     21450
weighted avg       0.65      0.68      0.63     21450



### HAY UNA MEJORA EN LOS SCORE

# RANDOM FOREST

In [44]:
#Creamos un random forest!
model = RandomForestClassifier(random_state=42, n_estimators=100,
                               class_weight="balanced", max_features="log2")
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_features='log2',
                       random_state=42)

In [45]:
y_test_pred = model.predict(X_test) #Prediccion en Test

## ACCURACY

In [46]:
###Completar
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_test_pred)

0.6206060606060606

## PRECISION SCORE

In [47]:
###Completar
from sklearn.metrics import precision_score
precision_score(y_test, y_test_pred, pos_label='1') 

0.43887443840151336

## RECALL SCORE

In [48]:
###Completar
from sklearn.metrics import recall_score
recall_score(y_test, y_test_pred, pos_label='1') 

0.5225225225225225

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.74      0.67      0.70     14346
           1       0.44      0.52      0.48      7104

    accuracy                           0.62     21450
   macro avg       0.59      0.60      0.59     21450
weighted avg       0.64      0.62      0.63     21450



# XGBOOST

In [50]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [51]:
#IMPORTAMOS BASE DE DATOS 1
bd=pd.read_excel(r'C:\Users\garro\OneDrive\Escritorio\DATA SCIENCE\TRABAJO PRACTICO\ViolenciaGenero2.3.xlsx', sheet_name='casos')

#CAMBIAMOS FORMATO DE LA COLUMNA FECHA A 'DATETIME'
bd['FECHA']=pd.to_datetime(bd.FECHA, errors='coerce')

#LIMPIEZA DE VALORES NULOS. REEMPLAZO POR MEDIA
bd['EDAD']= bd['EDAD'].fillna(method='bfill')

#REMOCION DE OUTLIERS
def remove_outliers(df, column_name):
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df = df.loc[(df[column_name]>=lower_bound) & (df[column_name] <= upper_bound)]
    return df
bd = remove_outliers(bd, 'EDAD')
bd.head(10)

,FECHA,PROVINCIA,GENERO_PERSONA_SIT_VIOLENCIA,EDAD,PAIS_NACIMIENTO,VIOLENCIA_FISICA,VIOLENCIA_PSICOLOGICA,VIOLENCIA_SEXUAL,VIOLENCIA_ECONOMICA,VIOLENCIA_SIMBOLICA,...,VIOLENCIA_LABORAL,VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA,VIOLENCIA_OTRAS,VINCULO_PERSONA_AGRESORA,GENERO_AGRESOR,CASO,TiPO_VIOLENCIA,LUGAR_VIOLENCIA,MOTIVO_VIOLENCIA,TIPOVIOLENCIA
0,2020-01-01,Tucumán,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
1,2020-01-01,Buenos Aires,Mujer,39.0,NaN,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
2,2020-01-01,Buenos Aires,Mujer,22.0,Argentina,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
3,2020-01-01,Buenos Aires,Mujer,53.0,NaN,0,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_PSICOLOGICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,0
4,2020-01-01,Buenos Aires,Mujer,38.0,Argentina,1,1,0,0,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
5,2020-01-01,Buenos Aires,Mujer,23.0,Argentina,1,1,0,1,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
6,2020-01-01,Santiago Del Estero,Mujer,25.0,Paraguaya,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
7,2020-01-01,Santa Fe,Mujer,24.0,Argentina,1,1,0,0,0,...,0,0,0,Ex pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_OTRAS,1
8,2020-01-01,Santa Fe,Mujer,21.0,Argentina,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1
9,2020-01-01,San Juan,Mujer,21.0,Argentina,1,1,0,1,0,...,0,0,0,Pareja,Varon,1,VIOLENCIA_FISICA,VIOLENCIA_DOMESTICA,VIOLENCIA_ECONOMICA,1


In [ ]:

# Extract feature and target arrays
# Paso 2: Dividir los datos en características (X) y variable objetivo (y)
X=bd.drop(['FECHA','VIOLENCIA_FISICA', 'VIOLENCIA_PSICOLOGICA', 'PROVINCIA','LUGAR_VIOLENCIA',
       'VIOLENCIA_SEXUAL', 'VIOLENCIA_ECONOMICA', 'VIOLENCIA_SIMBOLICA', 'EDAD',
       'VIOLENCIA_DOMESTICA', 'VIOLENCIA_INSTITUCIONAL', 'VIOLENCIA_LABORAL',
       'VIOLENCIA_CONTRA_LIBERTAD_REPRODUCTIVA', 'TIPOVIOLENCIA','TiPO_VIOLENCIA', 'VIOLENCIA_OTRAS','GENERO_PERSONA_SIT_VIOLENCIA', 'PAIS_NACIMIENTO', 'GENERO_AGRESOR', 'CASO'], axis=1)
y=bd['TIPOVIOLENCIA']
X

In [ ]:
import numpy as np

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [ ]:
# Paso 2: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Paso 3: Configurar los parámetros del modelo
params = {
    'objective': 'binary:logistic',  # Función objetivo para la clasificación binaria
    'eval_metric': 'logloss',  # Métrica de evaluación
    'eta': 0.1,  # Tasa de aprendizaje
    'max_depth': 3,  # Profundidad máxima de los árboles
    'subsample': 0.8,  # Proporción de muestras utilizadas en cada árbol
    'colsample_bytree': 0.8  # Proporción de características utilizadas en cada árbol
}

In [ ]:
# Paso 4: Entrenar el modelo
#dtrain = xgb.DMatrix(X_train, label=y_train)  # Convertir los datos a una matriz especializada para XGBoost
dtrain = xgb.DMatrix(X_train, y_train, enable_categorical=True)
model = xgb.train(params, dtrain, num_boost_round=100)


#dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
# Paso 5: Realizar predicciones en el conjunto de prueba
dtest = xgb.DMatrix(X_test, enable_categorical=True)
y_pred = model.predict(dtest)
y_pred_binary = [round(value) for value in y_pred]  # Convertir las probabilidades en etiquetas binarias


## ACCURACY

In [ ]:

# Paso 6: Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred_binary)
print("Precisión del modelo: %.2f%%" % (accuracy * 100.0))

## PRECISION SCORE

In [ ]:
###Completar
from sklearn.metrics import precision_score
precision_score(y_test, y_pred_binary) 

## RECALL SCORE

In [ ]:
###Completar
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_binary) 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_binary))